In [ ]:
import os
from DatasetParse import data_spilt, ReadH5Pkld, get_loader
from TrainInference import inference_net, inference_net_lesion, count_unique_contours
from monai.transforms import *

from monai.losses import DiceLoss, FocalLoss, DiceFocalLoss
from monai.networks.nets.swin_unetr import SwinUNETR
from torch.optim import AdamW

import torch

import matplotlib.pyplot as plt

import pandas as pd


use_dwi = False
model_path = 'archive_models_325/t2_adc_swin.ptm'
device='cpu'
base_dir = '/raid/candi/xiangcen/data_all_modality__update_crop'

if use_dwi:
    base_dir = '/raid/candi/xiangcen/data_all_modality_update_crop_three'
train_list, inference_list = data_spilt(base_dir, 745, seed=325)








inference_transform = ReadH5Pkld()



inference_loader = get_loader(inference_list, inference_transform, batch_size=1)
in_chan = 3
if use_dwi:
    in_chan = 4
model = SwinUNETR(img_size=(128, 128, 64), in_channels=in_chan, out_channels=3, downsample='mergingv2', use_v2=True)
model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
model.eval()







def post_process(x: torch.tensor):
    
    x = torch.argmax(x, dim=1)
    return x.to(float)

def inference_post_process(o):
    o = post_process(o) # make sure all value ins between 0-1
    return o # just get the 


# remember the loader should drop the last batch to prevent differenct sequence number in the last batch
model.to(device)
model.eval()


TP_gt, FP_gt = 0, 0
TP_pred, FP_pred = 0, 0

for batch in inference_loader:
    # print(batch["patient_name"], batch["patient_uid"])
    img, label = batch["img"].to(device), batch["label"].to(device)
    tumor = (label != 0).float()
    img = torch.cat([
        img,
        tumor
    ], dim=1)
    
    _, num_unique = count_unique_contours(label)
    print(num_unique)
    
    
    # forward pass
    output = model(img)
    output = inference_post_process(output)
    
    
    priads_sheet = pd.read_excel('Target-Data_2019-12-05-2.xlsx')
    
    



    patient_name, patient_uid = batch['patient_name'][0], batch['patient_uid']
    for uid in patient_uid:
        filtered_df = priads_sheet[priads_sheet['Patient ID'] == patient_name]
        filtered_df = filtered_df[filtered_df['seriesInstanceUID_MR'] == uid]
        filtered_df = filtered_df.drop_duplicates(subset='ROI Volume (cc)')
        print(patient_name, uid)
        print(filtered_df['UCLA Score (Similar to PIRADS v2)'])
    

    
    
    
    
    
    
    break
output = output.cpu().detach().numpy()
label = label.cpu().detach().numpy()
img = img.cpu().detach().numpy()

In [ ]:
rows, cols = 8, 8
fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
axes = axes.flatten()  # Flatten to easily index axes

for i in range(rows * cols):

    # axes[i].imshow(img[0, 1, :, :, i], cmap='gray')  # Adjust cmap if needed
    axes[i].imshow(output[0, :, :, i] == 1, cmap='gray')  # Adjust cmap if needed
    axes[i].contour(label[0, 0, :, :, i] == 1, levels=[0.5], colors='red')
    axes[i].contour(label[0, 0, :, :, i] == 2, levels=[0.5], colors='green')
    axes[i].axis('off')  # Hide axes

plt.tight_layout()
plt.show()

In [ ]:
rows, cols = 8, 8
fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
axes = axes.flatten()  # Flatten to easily index axes

for i in range(rows * cols):

    axes[i].imshow(img[0, 0, :, :, i], cmap='gray')  # Adjust cmap if needed
    axes[i].contour(output[0, :, :, i] == 1, linewidths=4, colors='orange')  # Adjust cmap if needed
    axes[i].contour(label[0, 0, :, :, i] == 1, levels=[0.5], colors='red')
    axes[i].contour(label[0, 0, :, :, i] == 2, levels=[0.5], colors='green')
    axes[i].axis('off')  # Hide axes

plt.tight_layout()
plt.show()

In [ ]:
# label = label.cpu()
# print(torch.unique(label))
# rows, cols = 8, 8
# fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
# axes = axes.flatten()  # Flatten to easily index axes
# for i in range(rows * cols):

#     axes[i].imshow(label[0, 0, :, :, i], cmap='gray')  # Adjust cmap if needed
#     axes[i].axis('off')  # Hide axes

# plt.tight_layout()
# plt.show()

In [ ]:
# img = img.cpu()
# # print(torch.unique(label))
# rows, cols = 8, 8
# fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
# axes = axes.flatten()  # Flatten to easily index axes
# for i in range(rows * cols):

#     axes[i].imshow(img[0, 1, :, :, i], cmap='gray')  # Adjust cmap if needed
#     axes[i].axis('off')  # Hide axes

# plt.tight_layout()
# plt.show()